# Intro to Object Detection Colab

Welcome to the object detection colab!  This demo will take you through the steps of running an "out-of-the-box" detection model on a collection of images.

## Imports and Setup

In [8]:
import os
import pathlib
import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
import pandas as pd
import time
from tqdm.notebook import tqdm

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

## Utilities

In [9]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def get_keypoint_tuples(eval_config):
  """Return a tuple list of keypoint edges from the eval config.
  
  Args:
    eval_config: an eval config containing the keypoint edges
  
  Returns:
    a list of edge tuples, each in the format (start, end)
  """
  tuple_list = []
  kp_list = eval_config.keypoint_edge
  for edge in kp_list:
    tuple_list.append((edge.start, edge.end))
  return tuple_list

### Build a detection model and load pre-trained model weights

This sometimes takes a little while, please be patient!

In [10]:
pipeline_config = "/home/ttran/projects/TFmodels2/models/research/object_detection/training/ssd_resnet152_v1_fpn_640x640_coco17_tpu-8.config"
model_dir = "/home/ttran/projects/TFmodels2/models/research/object_detection/ssd_resnet152_v1_fpn_640x640_coco17_tpu-8_3"

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(os.path.join(model_dir, 'ckpt-26')).expect_partial()

def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

detect_fn = get_model_detection_function(detection_model)

2024-12-11 13:57:07.585320: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-11 13:57:07.586904: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-11 13:57:07.609160: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

# Load label map data (for plotting).

Label maps correspond index numbers to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine.

In [11]:
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

### Putting everything together!

Run the below code which loads an image, runs it through the detection model and visualizes the detection results, including the keypoints.

Note that this will take a long time (several minutes) the first time you run this code due to tf.function's trace-compilation --- on subsequent runs (e.g. on new images), things will be faster.

Here are some simple things to try out if you are curious:
* Try running inference on your own images (local paths work)
* Modify some of the input images and see if detection still works.  Some simple things to try out here (just uncomment the relevant portions of code) include flipping the image horizontally, or converting to grayscale (note that we still expect the input image to have 3 channels).
* Print out `detections['detection_boxes']` and try to match the box locations to the boxes in the image.  Notice that coordinates are given in normalized form (i.e., in the interval [0, 1]).
* Set min_score_thresh to other values (between 0 and 1) to allow more detections in or to filter out more detections.

Note that you can run this cell repeatedly without rerunning earlier cells.


In [58]:
#image_dir = "/mnt/sda1/Backup/heif_lite"
image_dir = "/home/ttran/projects/TFmodels2/models/research/object_detection/test_images"
#image_path = os.path.join(image_dir, '0a00b11de9ad098befcd6543625b311a9a90ca80_lite.jpg')

result = []
label_id_offset = 1
timestamp = time.time()

def run_inference(directory):
    for filename in tqdm(os.listdir(directory), desc="Processing"):
        _result = {}
        file_path = os.path.join(directory, filename) 
        start_time = time.time()
        image_np = load_image_into_numpy_array(file_path)
        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
        detections, predictions_dict, shapes = detect_fn(input_tensor)
        # Class result
        pred_class = (detections['detection_classes'][0].numpy() + label_id_offset).astype(int)
        pred_score = detections['detection_scores'][0].numpy()
        end_time = time.time()
        _result['name'] = filename
        _result['timestamp'] = timestamp
        _result['inferred'] = "x"
        _result['time(ms)'] = round((end_time - start_time)*1000,2)
        for i in range(len(pred_class)):
            _result['pred_' + str(i+1)] = category_index[pred_class[0]]['name']
            _result['pred_' + str(i+1) + '_score'] = pred_score[0]
        print(f"Finished {filename} in {_result['time(ms)']} ms")
        result.append(_result)
    return result

result = run_inference(image_dir)
result_df = pd.DataFrame(result)
result_df.to_parquet(f"/mnt/sda1/Backup/result/result_{timestamp}.parquet")


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Finished 0a7dd59bb60415e67535a5b0b1c37c79b5fcdcf9.jpg in 3067.12 ms
Finished 0a7dd59bb60415e67535a5b0b1c37c79b5fcdcf9_lite_lite.jpg in 144.8 ms
Finished 0a1d30b3fe35b24a4c4cf538a1969696dc53ae01_lite.jpg in 142.51 ms
Finished 0a00b11de9ad098befcd6543625b311a9a90ca80_lite.jpg in 147.85 ms
Finished 0a00b11de9ad098befcd6543625b311a9a90ca80.jpg in 2771.06 ms
Finished 0a7dd59bb60415e67535a5b0b1c37c79b5fcdcf9_lite.jpg in 137.35 ms
Finished 0a1d30b3fe35b24a4c4cf538a1969696dc53ae01.jpg in 2712.59 ms


In [ ]:
num_pred = 100
list_of_pred = []
list_of_score=[]
col_to_drop = []
for i in range(num_pred):
    pred_col = "pred_" + str(i + 1)
    list_of_pred.append(pred_col)
    list_of_score = "pred_" + str(i + 1) + "_score"
col_to_drop.append(
col_to_drop.append(list_of_pred)

In [89]:
col_to_drop

[['name', 'timestamp', 'inferred', 'time(ms)'],
 ['pred_1',
  'pred_2',
  'pred_3',
  'pred_4',
  'pred_5',
  'pred_6',
  'pred_7',
  'pred_8',
  'pred_9',
  'pred_10',
  'pred_11',
  'pred_12',
  'pred_13',
  'pred_14',
  'pred_15',
  'pred_16',
  'pred_17',
  'pred_18',
  'pred_19',
  'pred_20',
  'pred_21',
  'pred_22',
  'pred_23',
  'pred_24',
  'pred_25',
  'pred_26',
  'pred_27',
  'pred_28',
  'pred_29',
  'pred_30',
  'pred_31',
  'pred_32',
  'pred_33',
  'pred_34',
  'pred_35',
  'pred_36',
  'pred_37',
  'pred_38',
  'pred_39',
  'pred_40',
  'pred_41',
  'pred_42',
  'pred_43',
  'pred_44',
  'pred_45',
  'pred_46',
  'pred_47',
  'pred_48',
  'pred_49',
  'pred_50',
  'pred_51',
  'pred_52',
  'pred_53',
  'pred_54',
  'pred_55',
  'pred_56',
  'pred_57',
  'pred_58',
  'pred_59',
  'pred_60',
  'pred_61',
  'pred_62',
  'pred_63',
  'pred_64',
  'pred_65',
  'pred_66',
  'pred_67',
  'pred_68',
  'pred_69',
  'pred_70',
  'pred_71',
  'pred_72',
  'pred_73',
  'pred_74',

In [107]:
view_result = pd.read_parquet(f"/mnt/sda1/Backup/result/result_1733951510.1521916.parquet")
#view_result.sort_values(by=['pred_1_score'], ascending=False).head(20)
view_result.to_dict(orient='records')


[{'name': '0a7dd59bb60415e67535a5b0b1c37c79b5fcdcf9.jpg',
  'timestamp': 1733951510.1521916,
  'inferred': 'x',
  'time(ms)': 3067.12,
  'pred_1': 'apr22',
  'pred_1_score': 0.8388247489929199,
  'pred_2': 'apr22',
  'pred_2_score': 0.8388247489929199,
  'pred_3': 'apr22',
  'pred_3_score': 0.8388247489929199,
  'pred_4': 'apr22',
  'pred_4_score': 0.8388247489929199,
  'pred_5': 'apr22',
  'pred_5_score': 0.8388247489929199,
  'pred_6': 'apr22',
  'pred_6_score': 0.8388247489929199,
  'pred_7': 'apr22',
  'pred_7_score': 0.8388247489929199,
  'pred_8': 'apr22',
  'pred_8_score': 0.8388247489929199,
  'pred_9': 'apr22',
  'pred_9_score': 0.8388247489929199,
  'pred_10': 'apr22',
  'pred_10_score': 0.8388247489929199,
  'pred_11': 'apr22',
  'pred_11_score': 0.8388247489929199,
  'pred_12': 'apr22',
  'pred_12_score': 0.8388247489929199,
  'pred_13': 'apr22',
  'pred_13_score': 0.8388247489929199,
  'pred_14': 'apr22',
  'pred_14_score': 0.8388247489929199,
  'pred_15': 'apr22',
  'pred

In [109]:
view_result2 = view_result.drop(list_of_pred, axis=1).drop(['name', 'timestamp', 'inferred', 'time(ms)'], axis=1)
view_result2

,pred_1_score,pred_2_score,pred_3_score,pred_4_score,pred_5_score,pred_6_score,pred_7_score,pred_8_score,pred_9_score,pred_10_score,...,pred_91_score,pred_92_score,pred_93_score,pred_94_score,pred_95_score,pred_96_score,pred_97_score,pred_98_score,pred_99_score,pred_100_score
0,0.838825,0.838825,0.838825,0.838825,0.838825,0.838825,0.838825,0.838825,0.838825,0.838825,...,0.838825,0.838825,0.838825,0.838825,0.838825,0.838825,0.838825,0.838825,0.838825,0.838825
1,0.855161,0.855161,0.855161,0.855161,0.855161,0.855161,0.855161,0.855161,0.855161,0.855161,...,0.855161,0.855161,0.855161,0.855161,0.855161,0.855161,0.855161,0.855161,0.855161,0.855161
2,0.255404,0.255404,0.255404,0.255404,0.255404,0.255404,0.255404,0.255404,0.255404,0.255404,...,0.255404,0.255404,0.255404,0.255404,0.255404,0.255404,0.255404,0.255404,0.255404,0.255404
3,0.804084,0.804084,0.804084,0.804084,0.804084,0.804084,0.804084,0.804084,0.804084,0.804084,...,0.804084,0.804084,0.804084,0.804084,0.804084,0.804084,0.804084,0.804084,0.804084,0.804084
4,0.782242,0.782242,0.782242,0.782242,0.782242,0.782242,0.782242,0.782242,0.782242,0.782242,...,0.782242,0.782242,0.782242,0.782242,0.782242,0.782242,0.782242,0.782242,0.782242,0.782242
5,0.854954,0.854954,0.854954,0.854954,0.854954,0.854954,0.854954,0.854954,0.854954,0.854954,...,0.854954,0.854954,0.854954,0.854954,0.854954,0.854954,0.854954,0.854954,0.854954,0.854954
6,0.259388,0.259388,0.259388,0.259388,0.259388,0.259388,0.259388,0.259388,0.259388,0.259388,...,0.259388,0.259388,0.259388,0.259388,0.259388,0.259388,0.259388,0.259388,0.259388,0.259388


In [110]:
view_result2.idxmax(axis=1)

0    pred_1_score
1    pred_1_score
2    pred_1_score
3    pred_1_score
4    pred_1_score
5    pred_1_score
6    pred_1_score
dtype: object

In [111]:
view_result[(view_result.pred_1_score > 0.5)]

,name,timestamp,inferred,time(ms),pred_1,pred_1_score,pred_2,pred_2_score,pred_3,pred_3_score,...,pred_96,pred_96_score,pred_97,pred_97_score,pred_98,pred_98_score,pred_99,pred_99_score,pred_100,pred_100_score
0,0a7dd59bb60415e67535a5b0b1c37c79b5fcdcf9.jpg,1.733952e+09,x,3067.12,apr22,0.838825,apr22,0.838825,apr22,0.838825,...,apr22,0.838825,apr22,0.838825,apr22,0.838825,apr22,0.838825,apr22,0.838825
1,0a7dd59bb60415e67535a5b0b1c37c79b5fcdcf9_lite_...,1.733952e+09,x,144.80,jun22,0.855161,jun22,0.855161,jun22,0.855161,...,jun22,0.855161,jun22,0.855161,jun22,0.855161,jun22,0.855161,jun22,0.855161
3,0a00b11de9ad098befcd6543625b311a9a90ca80_lite.jpg,1.733952e+09,x,147.85,jun22,0.804084,jun22,0.804084,jun22,0.804084,...,jun22,0.804084,jun22,0.804084,jun22,0.804084,jun22,0.804084,jun22,0.804084
4,0a00b11de9ad098befcd6543625b311a9a90ca80.jpg,1.733952e+09,x,2771.06,jun22,0.782242,jun22,0.782242,jun22,0.782242,...,jun22,0.782242,jun22,0.782242,jun22,0.782242,jun22,0.782242,jun22,0.782242
5,0a7dd59bb60415e67535a5b0b1c37c79b5fcdcf9_lite.jpg,1.733952e+09,x,137.35,jun22,0.854954,jun22,0.854954,jun22,0.854954,...,jun22,0.854954,jun22,0.854954,jun22,0.854954,jun22,0.854954,jun22,0.854954
